In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from environment import NYCEnv

from stable_baselines import ACKTR
from stable_baselines.common.cmd_util import make_vec_env

In [3]:
env = NYCEnv()

In [4]:
from stable_baselines.common.env_checker import check_env
check_env(env, warn=True)

In [5]:
obs = env.reset()
env.render()

print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())
print(obs)
print('==========')

n_steps = 100
for step in range(n_steps):
    print("Step {}".format(step + 1))
    print("Action {}".format(obs[0]))
    obs, reward, done, info = env.step(obs[0])
    env.render()
    print('==========')
    if done:
        print("done")
        break

Current taxi zone: 70, time: 32, reward: 0.00
Box(1, 263, (2,), int32)
Discrete(264)
192
[70 32]
Step 1
Action 70
Current taxi zone: 163, time: 31.0, reward: -2.17
Step 2
Action 163.0
Current taxi zone: 237, time: 31.0, reward: 2.87
Step 3
Action 237.0
Current taxi zone: 237, time: 31.0, reward: 10.28
Step 4
Action 237.0
Current taxi zone: 161, time: 32.0, reward: 16.53
Step 5
Action 161.0
Current taxi zone: 161, time: 32.0, reward: 21.12
Step 6
Action 161.0
Current taxi zone: 138, time: 34.0, reward: 49.84
Step 7
Action 138.0
Current taxi zone: 100, time: 46.0, reward: 84.22
Step 8
Action 100.0
Current taxi zone: 68, time: 47.0, reward: 89.72
Step 9
Action 68.0
Current taxi zone: 141, time: 49.0, reward: 104.21
Step 10
Action 141.0
Current taxi zone: 161, time: 50.0, reward: 113.65
Step 11
Action 161.0
Current taxi zone: 148, time: 52.0, reward: 129.22
Step 12
Action 148.0
Current taxi zone: 140, time: 53.0, reward: 145.60
Step 13
Action 140.0
Current taxi zone: 237, time: 54.0, rewar

In [12]:
env = make_vec_env(lambda: env, n_envs=1)

In [13]:
model = ACKTR('MlpPolicy', env, verbose=1).learn(50000)

In [21]:
# Test the trained agent
obs = env.reset()
print("obs=", obs)
print('==========')
n_steps = 20
for step in range(n_steps):
    action, _ = model.predict(obs, deterministic=True)
    print("Step {}".format(step + 1))
    print("Action: ", action)
    obs, reward, done, info = env.step(action)
    print('obs=', obs, 'reward=', reward, 'done=', done)
    env.render(mode='console')
    print('==========')
    if done:
        # Note that the VecEnv resets automatically
        # when a done signal is encountered
        print("Goal reached!", "reward=", reward)
        break

obs= [[218  32]]
Step 1
Action:  [128]
obs= [[61 32]] reward= [-10000.] done= [ True]
Current taxi zone: 61, current time: 32, current reward: 0.00
Goal reached! reward= [-10000.]
